In [2]:
import sys
sys.path.append(r"C:\Users\NLin\OneDrive - AQUILA Commercial\Documents\GitHub\AquilaResearch")
import aquila_tools

import pandas as pd


1. Load data
2. Build Figures
3. Save as HTML String
4. Save to Charts
5. Upload to Github

In [3]:
conn, cursor = aquila_tools.initialize_connection()

cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
tables = cursor.fetchall()
print("Available tables:", [table[0] for table in tables])


Available tables: ['market_tables_office', 'quarterly_report_data_office', 'issues_stats_industrial', 'issues_stats_office', 'building_list_industrial', 'market_tables_office_survey_overlay', 'quarterly_report_data_industrial', 'v_property_changes_industrial', 'v_property_changes_office', 'inventory_retail', 'key_survey_report_industrial', 'market_tables_industrial', 'building_list_office', 'stats_industrial', 'key_survey_report_office', 'stats_office', 'survey_office', 'survey_industrial', 'inventory_office', 'inventory_industrial', 'audit_log']


In [ ]:


# Plotly styling module for Aquila-style charts
import plotly.express as px
from datetime import datetime

AQUILA_COLORS = ["#00325a", "#e6b40a", "#8c8c8c", "#18a0CF", "#D05325"]
AQUILA_FONT = "Futura LT Pro, Futura, Arial, sans-serif"

def aquila_styled_line_chart(
    df,
    x,
    y,
    color=None,
    facet_row=None,
    title="",

    height=800,
    date_annotation=False,
):
    """
    Build a Plotly line chart with Aquila style settings.

    Parameters
    ----------
    df : pd.DataFrame
        Source data.
    x, y : str
        Column names.
    color : str, optional
        Name of column to group/color lines.
    facet_row : str, optional
        Row facet column.
    title : str
        Chart title.
    height : int
        Chart height.
    date_annotation : bool
        If True, adds an annotation with today's date.
    """
    fig = px.line(
        df,
        x=x,
        y=y,
        color=color,
        facet_row=facet_row,
        title=title,
        color_discrete_sequence=AQUILA_COLORS,
    )
    # No vertical lines: skip vline_shapes and related logic
    vline_shapes = []

    # Borders: Only show left and bottom (x and y axis lines), hide others
    layout_dict = dict(
        height=height,
        plot_bgcolor="white",
        paper_bgcolor="white",
        font=dict(family=AQUILA_FONT, color="#00325a"),
        title_font_family=AQUILA_FONT,
        legend=dict(
            title_font_family=AQUILA_FONT,
            orientation="h",
            yanchor="bottom",
            y=-0.25,
            xanchor="center",
            x=0.5,
        ),
        margin=dict(b=100),
        xaxis_tickangle=90,
        xaxis=dict(
            showline=True,
            linecolor="#e9e9ea",
            linewidth=.5,
            mirror=False,
            showgrid=False,   # Hide background grid
            zeroline=False,
        ),
        yaxis=dict(
            showline=True,
            linecolor="#e9e9ea",
            linewidth=.5,
            mirror=False,
            showgrid=True,
            gridcolor="#e9e9ea",
            zeroline=False,
        ),
        shapes=vline_shapes,
    )

    # If annotation with date
    if date_annotation:
        today_str = datetime.today().strftime("%Y-%m-%d")
        footer_text = f"Date Created/Updated: {today_str}"
        layout_dict["annotations"] = [
            dict(
                text=footer_text,
                xref="paper",
                yref="paper",
                x=1,
                y=-0.25,
                showarrow=False,
                font=dict(
                    size=10,
                    family=AQUILA_FONT,
                    color="#8c8c8c",
                ),
                xanchor="right",
                yanchor="bottom",
            )
        ]

    fig.update_layout(**layout_dict)
    return fig

# Example usage of the template:
# Query every row from market_tables_industrial
query = "SELECT * FROM market_tables_industrial;"
df = pd.read_sql(query, conn)

fig = aquila_styled_line_chart(
    df[df["property_type"] == "Industrial"],
    x="quarter",
    y="total_vacancy_rate",
    color="submarket_name",
    title="Vacancy Rate by Quarter (Industrial Market) by Submarket and Property Type"
)
fig.update_yaxes(tickformat=".0%")  # Show y-axis as percentage
fig.show()


C:\Users\NLin\AppData\Local\Temp\ipykernel_34416\3457275114.py:120: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Upload to Git

In [36]:
import aquila_graphing_tools
fig.write_html("charts/vacancy_rate_industrial.html")
aquila_graphing_tools.commit_and_push_all("updated name for vacancy rate")


# Example usage:
# commit_and_push_all("Update readme instructions")
